In [1]:
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
#from sklearn.ensemble import RandomForestClassifier
from models import *
from get_feature import *
from Generator0 import *
import pickle as pk
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
import datetime
from IPython.display import clear_output
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
sys.path.insert(0,'lucashnegri-peakutils-51a679cd8428')

from keras.preprocessing import sequence
import peakutils
from scipy import special
import scipy.io as sio
from keras.preprocessing.sequence import pad_sequences
import peakutils
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint

In [2]:
    train_folder =  '/home/jk21/Downloads/Data/data/murmur/train'
    test_folder = '/home/jk21/Downloads/Data/data/murmur/test'
    patient_files_trn = find_patient_files(train_folder)
    patient_files_test = find_patient_files(test_folder)
    
    per_sec=4000
    winlen = 512
    hoplen = 256
    nmel = 120 
    nsec = 20 
    trim = 1 
    use_mel=True
    use_cqt = False 
    use_stft = False
    ord1 = True #np.random.choice([True,False])
    maxlen1 = 120000
    min_dist = np.random.choice(list(range(10,2000 ,10)))
    max_interval_len = np.random.choice(list(range(100,300 ,10)))
    
    params_feature = {'samp_sec': nsec,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': hoplen,
                  'win_length': winlen,
                  'n_mels': nmel,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10,
                  'maxlen1': maxlen1,
                  'min_dist':min_dist,
                  'max_interval_len' : max_interval_len,
                  'trim' :1,
                  'use_mel' : use_mel,
                  'use_cqt' : use_cqt,
                  'use_stft' : use_stft,
                  'per_sec' : per_sec}
    
    model_folder = 'lcnn2'

In [3]:

    
    def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
        
        if e < start:
            return lr_start
        elif e > end:
            return lr_end

        middle = (start + end) / 2
        s = lambda x: 1 / (1 + np.exp(-x))

        return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

    if ord1 :
        features_trn, mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape = get_features_3lb_all_ord_rr(train_folder, patient_files_trn[:64], **params_feature)
    else :
        features_trn, mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape = get_features_3lb_all(train_folder, patient_files_trn, **params_feature)

  0%|          | 0/64 [00:00<?, ?it/s]/home/jk21/anaconda3/envs/new_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jk21/anaconda3/envs/new_env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 17%|█▋        | 11/64 [00:12<00:52,  1.00it/s]/home/jk21/anaconda3/envs/new_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jk21/anaconda3/envs/new_env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 19%|█▉        | 12/64 [00:14<00:55,  1.07s/it]/home/jk21/anaconda3/envs/new_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.

melspec:  120 313
cqt:  1 1
stft:  1 1


In [5]:


params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 120,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'maxlen1': 120000,
 'min_dist': 1990,
 'max_interval_len': 150,
 'trim': 1,
 'use_mel': True,
 'use_cqt': False,
 'use_stft': False,
 'per_sec': 4000}

In [20]:
    mm_weight = 3 #np.random.choice([2,3,4,5])
    oo_weight = 3 #np.random.choice([2,3,4,5,6])
    ord1 = True #np.random.choice([True,False])
    mm_mean = False #np.random.choice([True,False])
    dp = 0 #np.random.choice([0, .1, .2, .3])
    fc = False #np.random.choice([True,False])
    interval_seq = np.random.choice([True,False])
    ext = False
    ext2 = True
    if interval_seq==True:
        
        model_index=np.random.choice([0,1])
        model_list = [['lcnn1','lcnn2'],['lcnn5','lcnn6']][model_index]
        m1,m2 = model_list
      
    if interval_seq==False:
        model_index=np.random.choice([0,1])
        model_list = [['lcnn3','lcnn4'],['lcnn7','lcnn8']][model_index]
        m1,m2 = model_list
    
    
    chaug = 10 #np.random.choice([0, 10])
    mixup = True #np.random.choice([True,False])
    cout = .8 #np.random.choice([0, 0.8])
    wunknown = 1 #np.random.choice([1, 0.7, .5, .2])
    n1 = 0 #np.random.choice([0,2])
    
    if n1 == 0 :
        ranfil = False
    else :
        ranfil = [n1, [18,19,20,21,22,23]]


In [21]:
m1

'lcnn5'

In [22]:
    nep = 5
    
    params_feature['ord1'] = ord1
    params_feature['mm_mean'] = mm_mean
    params_feature['dp'] = dp
    params_feature['fc'] = fc
    params_feature['ext'] = ext
    params_feature['oo_weight'] = oo_weight
    params_feature['mm_weight'] = mm_weight
    params_feature['chaug'] = chaug
    params_feature['cout'] = cout
    params_feature['wunknown'] = wunknown
    params_feature['mixup'] = mixup
    params_feature['n1'] = n1
    params_feature['use_interval_seq'] = interval_seq
    params_feature['m_name1'] = m1
    params_feature['m_name2'] = m2
    mel_input_shape = features_trn['mel1'].shape[1:]
    cqt_input_shape = features_trn['cqt1'].shape[1:]
    stft_input_shape = features_trn['stft1'].shape[1:]
    interval_input_shape= features_trn['interval'].shape[1:]
    interval_mean_input_shape = (1,)

In [23]:
    if m1 =='lcnn1':
        model1=LCNN_seq_mean_model_1(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape, ord1= ord1,dp = dp, fc = fc, ext = False, ext2=True)
    if m2 =='lcnn2':
        model2=LCNN_seq_mean_model_2(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape, dp = dp, fc = fc, ext = True,ext2=False)
    if m1 =='lcnn3':
        model1=LCNN_mean_model_1(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape,ord1= ord1,dp = dp, fc = fc, ext = False, ext2=True)
    if m2 =='lcnn4':
        model2=LCNN_mean_model_2(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape,dp = dp, fc = fc, ext = True,ext2=False)    

    if m1 =='lcnn5':
        model1=LCNN_seq_mean_jk_model_1(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape, ord1= ord1,dp = dp, fc = fc, ext = False, ext2=True)
    if m2 =='lcnn6':
        model2=LCNN_seq_mean_jk_model_2(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape, dp = dp, fc = fc, ext = True,ext2=False)
    if m1 =='lcnn7':
        model1=LCNN_mean_jk_model_1(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape,ord1= ord1,dp = dp, fc = fc, ext = False, ext2=True)
    if m2 =='lcnn8' :
        model2=LCNN_mean_jk_model_2(mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape,dp = dp, fc = fc, ext = True,ext2=False)    

In [24]:
m2

'lcnn6'

In [25]:
    n_epoch = nep
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
    batch_size = 64

    if mixup :
        beta_param = .7
    else :
        beta_param = 0



    params = {'batch_size': batch_size,
              #          'input_shape': (100, 313, 1),
              'shuffle': True,
              'chaug': chaug,
              'beta_param': beta_param,
              'cout': cout
    #              'mixup': mixup,
              #          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
              #          'highpass': [.5, [78,79,80,81,82,83,84,85]]
    #              'ranfilter2' : [3, [18,19,20,21,22,23]]
              #           'dropblock' : [30, 100]
              #'device' : device
    }

    if mixup :
        params['mixup'] = mixup
        params['ranfilter2'] = ranfil
    else :
        params['cutout'] = cout

    params_no_shuffle = {'batch_size': batch_size,
                         #          'input_shape': (100, 313, 1),
                         'shuffle': False,
                         'beta_param': 0.7,
                         'mixup': False
                         #'device': device
    }



    if ord1 :
        class_weight = {0: mm_weight, 1: 1.}
    else :
        class_weight = {0: mm_weight, 1: wunknown, 2:1.}


    if mixup :


        TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], 
                                  features_trn['preg'], features_trn['loc'],
                                  features_trn['mel1'], features_trn['cqt1'],features_trn['stft1'],
                                  features_trn['interval_mean'],features_trn['interval']],features_trn['mm_labels'],**params)()

        model1.fit(TrainDGen_1,
#                    validation_data = ([features_test['age'],features_test['sex'], features_test['hw'],                                   
#                                        features_test['preg'], features_test['loc'],
#                                        features_test['interval'],
#                                        features_test['mel1'],
#                                        features_test['cqt1'],
#                                        features_test['stft1'],features_test['interval_mean']],features_test['mm_labels']),

                   callbacks=[lr],
                   steps_per_epoch=np.ceil(len(features_trn['mm_labels'])/batch_size),
                   class_weight=class_weight, 
                   epochs = n_epoch)
    
    
 
    n_epoch = nep
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))

    batch_size = 64

    if mixup :
        beta_param = .7
    else :
        beta_param = 0

    params = {'batch_size': batch_size,
              #          'input_shape': (100, 313, 1),
              'shuffle': True,
              'chaug': 0,
              'beta_param': beta_param,
              'cout': cout,
    #              'mixup': True,
              #          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
    #            'highpass': [.5, [78,79,80,81,82,83,84,85]],
    #              'ranfilter2' : [3, [18,19,20,21,22,23]]
            #           'dropblock' : [30, 100]
              #'device' : device
    }



    if mixup :
        params['mixup'] = mixup
        params['ranfilter2'] = ranfil
    else :
        params['cutout'] = cout


    params_no_shuffle = {'batch_size': batch_size,
                         #          'input_shape': (100, 313, 1),
                         'shuffle': False,
                         'beta_param': 0.7,
                         'mixup': False
                         #'device': device
    }




    if mixup :


        class_weight = {0: oo_weight, 1: 1.}
        TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], 
                                  features_trn['preg'], features_trn['loc'],
                                  features_trn['mel1'], features_trn['cqt1'],features_trn['stft1'],
                                  features_trn['interval_mean'],features_trn['interval']],features_trn['out_labels'],**params)()

        model2.fit(TrainDGen_1,
#                    validation_data = ([features_test['age'],features_test['sex'], features_test['hw'],                                   
#                                        features_test['preg'], features_test['loc'],features_test['interval'],
#                                        features_test['mel1'],
#                                        features_test['cqt1'],
#                                        features_test['stft1'],features_test['interval_mean']],features_test['out_labels']), 
                   callbacks=[lr],
            
                   steps_per_epoch=np.ceil(len(features_trn['out_labels'])/batch_size),
                   class_weight=class_weight, 
                   epochs = n_epoch)
    
    params_feature['mel_shape'] = mel_input_shape
    params_feature['cqt_shape'] = cqt_input_shape
    params_feature['stft_shape'] = stft_input_shape
    params_feature['interval_shape'] = interval_input_shape
    params_feature['interval_mean_shape'] = interval_mean_input_shape
    params_feature['max_interval_len'] = max_interval_len
    
    
    import pickle as pk
    def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
        os.makedirs(model_folder, exist_ok=True)
        info_fnm = os.path.join(model_folder, 'desc.pk')
        filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
        filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
        model1.save(filename1)
        model2.save(filename2)
        params_feature['model1'] = m_name1
        params_feature['model2'] = m_name2
        params_feature['model_fnm1'] = filename1
        params_feature['model_fnm2'] = filename2
        with open(info_fnm, 'wb') as f:
            pk.dump(params_feature, f, pk.HIGHEST_PROTOCOL)
        return 1

        with open(info_fnm, 'wb') as f:
            pk.dump(params_feature, f, pk.HIGHEST_PROTOCOL)
        return 1
    
    save_challenge_model(model_folder, model1, model2, m_name1 = m1, m_name2 = m2, param_feature = params_feature)

Epoch 1/5
4/4 [==============================] - 15s 3s/step - loss: 1.3134 - accuracy: 0.4102 - auc: 0.4184 - lr: 9.9851e-04
Epoch 2/5
4/4 [==============================] - 10s 2s/step - loss: 1.2201 - accuracy: 0.4336 - auc: 0.4080 - lr: 9.8036e-04
Epoch 3/5
4/4 [==============================] - 10s 3s/step - loss: 1.2278 - accuracy: 0.5273 - auc: 0.5195 - lr: 7.8798e-04
Epoch 4/5
4/4 [==============================] - 10s 2s/step - loss: 1.2494 - accuracy: 0.4414 - auc: 0.4622 - lr: 2.2202e-04
Epoch 5/5
4/4 [==============================] - 10s 3s/step - loss: 1.1266 - accuracy: 0.4102 - auc: 0.4970 - lr: 2.9642e-05
Epoch 1/5
4/4 [==============================] - 16s 3s/step - loss: 46.1701 - accuracy: 0.1680 - auc: 0.1267 - lr: 9.9851e-04
Epoch 2/5
4/4 [==============================] - 10s 3s/step - loss: 36.8332 - accuracy: 0.1953 - auc: 0.1466 - lr: 9.8036e-04
Epoch 3/5
4/4 [==============================] - 11s 3s/step - loss: 30.1162 - accuracy: 0.1406 - auc: 0.1373 - lr: 

1

In [36]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    params_feature['model1'] = m_name1
    params_feature['model2'] = m_name2
    params_feature['model_fnm1'] = filename1
    params_feature['model_fnm2'] = filename2
    with open(info_fnm, 'wb') as f:
        pk.dump(params_feature, f, pk.HIGHEST_PROTOCOL)
    return 1

    with open(info_fnm, 'wb') as f:
        pk.dump(params_feature, f, pk.HIGHEST_PROTOCOL)
    return 1
    
save_challenge_model(model_folder, model1, model2, m_name1 = m1, m_name2 = m2, param_feature = params_feature)


1

In [37]:
save_challenge_model(model_folder, model1, model2, m_name1 = m1, m_name2 = m2, param_feature = params_feature)


1

In [38]:
model_folder

'lcnn2'

In [39]:
  def load_challenge_model(model_folder, verbose):
        info_fnm = os.path.join(model_folder, 'desc.pk')
        with open(info_fnm, 'rb') as f:
            info_m = pk.load(f)

        return info_m
    
    ########################RunModel

In [40]:
model = load_challenge_model(model_folder, verbose=1)

In [41]:
model

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 120,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'maxlen1': 120000,
 'min_dist': 210,
 'max_interval_len': 170,
 'trim': 1,
 'use_mel': True,
 'use_cqt': False,
 'use_stft': False,
 'use_interval_seq': True,
 'per_sec': 4000,
 'ord1': True,
 'mm_mean': False,
 'dp': 0,
 'fc': False,
 'ext': False,
 'oo_weight': 3,
 'mm_weight': 3,
 'chaug': 10,
 'cout': 0.8,
 'wunknown': 1,
 'mixup': True,
 'n1': 0,
 'm_name1': 'lcnn1',
 'm_name2': 'lcnn2',
 'mel_shape': (120, 313, 1),
 'cqt_shape': (1, 1, 1),
 'stft_shape': (1, 1, 1),
 'interval_shape': (170, 1),
 'interval_mean_shape': (1,),
 'model1': 'lcnn1',
 'model2': 'lcnn2',
 'model_fnm1': 'lcnn2/lcnn1_model1.hdf5',
 'model_fnm2': 'lcnn2/lcnn2_model2.hdf5'}

In [42]:
    def run_challenge_model(model, data, recordings, verbose):
        
        murmur_classes = ['Present', 'Unknown', 'Absent']
        outcome_classes = ['Abnormal', 'Normal']

    ##########################################################################    
        if model['model1'] == 'lcnn1' :
            model1 = get_LCNN_o_1_dr_rr(model['mel_shape'],model['cqt_shape'],model['stft_shape'], model['interval_shape'],
                                     use_mel = model['use_mel'],use_cqt = model['use_cqt'], 
                                     use_stft = model['use_stft'],
                                     ord1 = model['ord1'], dp = model['dp'], fc = model['fc'], 
                                     ext = False, ext2= True,use_interval_seq=model['use_interval_seq'])  
        if model['model2'] == 'lcnn2' :
            model2 = get_LCNN_2_dr_rr(model['mel_shape'],model['cqt_shape'],model['stft_shape'], 
                                       model['interval_shape'],
                                       use_mel = model['use_mel'],use_cqt = model['use_cqt'], 
                                       use_stft = model['use_stft'],
                                       dp = model['dp'], fc = model['fc'], 
                                       ext = True, ext2=False,use_interval_seq=model['use_interval_seq']) 

        
        # if model['model1'] == 'lcnn3' :
        #     model1 = get_LCNN_pi_seq_mean_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], 
        #                                       model['interval_shape'],model['interval_mean_shape'],
        #                                       use_mel = model['use_mel'],use_cqt = model['use_cqt'], 
        #                                       use_stft = model['use_stft'],
        #                                       ord1 = model['ord1'], dp = model['dp'], fc = model['fc'], 
        #                                       ext = False, ext2= True) 
            
        # if model['model2'] == 'lcnn4' :
        #     model2 = get_LCNN_pi_seq_mean_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], 
        #                                         model['interval_shape'], model['interval_mean_shape'],
        #                                         use_mel = model['use_mel'],use_cqt = model['use_cqt'], 
        #                                         use_stft = model['use_stft'],ord1 = model['ord1'], 
        #                                         dp = model['dp'], fc = model['fc'], 
        #                                     ext = True, ext2=False)

#         if model['model1'] == 'lcnn5' :
#             model1 = get_LCNN_o_4_dr_interval_mean(model['mel_shape'],model['cqt_shape'],model['stft_shape'], 
#                                                    model['interval_shape'],model['interval_mean_shape'],
#                                                    use_mel = model['use_mel'],use_cqt = model['use_cqt'], 
#                                                    use_stft = model['use_stft'], ord1 = model['ord1'], 
#                                                    dp = model['dp'], fc = model['fc'], ext = model['ext'])  
            
#         if model['model2'] == 'lcnn6' :
#             model2 = get_LCNN_o_4_dr_1_interval_mean(model['mel_shape'],model['cqt_shape'],model['stft_shape'], 
#                                                      model['interval_shape'],model['interval_mean_shape'],
#                                                      use_mel = model['use_mel'],use_cqt = model['use_cqt'], 
#                                                      use_stft = model['use_stft'],ord1 = model['ord1'], 
#                                                      dp = model['dp'], fc = model['fc'], ext = model['ext'])
        
        

        model1.load_weights(model['model_fnm1'])
        model2.load_weights(model['model_fnm2'])
    ############################################################################ 




        # Load features.
        features = get_feature_one(data, verbose = 0)

        samp_sec = model['samp_sec'] 
        pre_emphasis = model['pre_emphasis']
        hop_length = model['hop_length']
        win_length = model['win_length']
        n_mels = model['n_mels']
        filter_scale = model['filter_scale']
        n_bins = model['n_bins']
        fmin = model['fmin']
        trim = model['trim']
        use_mel = model['use_mel']
        use_cqt = model['use_cqt']
        use_stft = model['use_stft']
        use_interval_seq = model['use_interval_seq']
        maxlen1 = model['maxlen1']
        min_dist = model['min_dist']
        per_sec = model['per_sec']

    #    use_raw = model['use_raw']

        max_interval_len = model['max_interval_len']


        # Load features.


        tmp_total_interval = [] 
        tmp_total_interval_mean = []

        features['interval'] = []
        features['interval_mean']=[]

        if use_interval_seq:
            

            for i in range(len(recordings)):
                datos=recordings[i]
                filtros=sio.loadmat('./Filters1')
                tmp_interval = []


                try:
                    X = datos
                    Fs= 4000
                    Fpa20=filtros['Fpa20'];			        # High pass filter
                    Fpa20=Fpa20[0];					# High pass filter
                    Fpb100=filtros['Fpb100'];		        # Low-pass Filter
                    Fpb100=Fpb100[0];				# Low-pass Filter
                    Xf=FpassBand(X,Fpa20,Fpb100); 	                # Apply a passband filter
                    Xf=vec_nor(Xf);			

        # Derivate of the Signal
                    dX=derivate(Xf);				# Derivate of the signal
                    dX=vec_nor(dX);					# Vector Normalizing
        # Square of the signal
                    dy=np.square(Xf);
                    dy=vec_nor(dy);

                    size=np.shape(Xf)				# Rank or dimension of the array
                    fil=size[0];					# Number of rows

                    positive=np.zeros((1,fil+1));                   # Initializating Positives Values Vector 
                    positive=positive[0];                           # Getting the Vector

                    points=np.zeros((1,fil));                       # Initializating the all Peak Points Vector
                    points=points[0];                               # Getting the point vector

                    peaks=np.zeros((1,fil));                        # Initializating the s1-s1 Peak Vector
                    peaks=peaks[0];                                 # Getting the point vector


                    for i in range(0,fil):
                        if dX[i]>0:
                            positive[i]=1;
                        else:
                            positive[i]=0;

                    for i in range(0,fil):
                        if (positive[i]==1 and positive[i+1]==0):
                            points[i]=Xf[i];
                        else:
                            points[i]=0;

                    indexes=peakutils.indexes(points,thres=0.5/max(points), min_dist=min_dist);
                    lenght=np.shape(indexes)			# Get the length of the index vector		
                    lenght=lenght[0];				# Get the value of the index vector

                    for i in range(0,lenght):
                        p=indexes[i];
                        peaks[p]=points[p];

                    n=np.arange(0,fil);

                    ############### interval distance ##############                   
                    tmp_peaks = np.diff(indexes)/4000

                    ############## interval sequence ###############
                    tmp_peaks_inteval = tmp_peaks


                    ########### interval distance mean ############
                    tmp_peaks = np.array(tmp_peaks)
                    tmp_peaks = np.mean(tmp_peaks)

                    if len(tmp_peaks_inteval) == 1:
                        tmp_peaks_inteval = np.zeros(1)

    #                         tmp_peaks_mean = np.mean(tmp_peaks)
    #                         tmp_peaks_std = np.std(tmp_peaks)

    #                         tmp_final = np.divide((tmp_peaks-tmp_peaks_mean),tmp_peaks_std)

                    tmp_peaks = np.nan_to_num(tmp_peaks)
    #                     print(tmp_final)
                    tmp_total_interval_mean.append(tmp_peaks)
                    tmp_total_interval.append(tmp_peaks_inteval)


                except:
                    print(i)
                    tmp_peaks = np.zeros(1)
                    tmp_peaks_inteval = np.zeros(1)
                    tmp_total_interval_mean.append(tmp_peaks)
                    tmp_total_interval.append(tmp_peaks_inteval)

        else :
            tmp_peaks = np.zeros(1)
            tmp_peaks_inteval = np.zeros(1)
            tmp_total_interval_mean.append(tmp_peaks)
            tmp_total_interval.append(tmp_peaks_inteval)      


        if use_interval_seq:


            ############ interval Sequence #################
            padded =pad_sequences(tmp_total_interval, maxlen=max_interval_len, dtype='float32', padding='post', truncating='post',
                                  value=0.0)

            for i in range(len(padded)):
                features['interval'].append(padded[i])
            for i in range(len(features['interval'])):
                features['interval'][i]= features['interval'][i].reshape(-1,1)
            features['interval']= np.array(features['interval'])


            features['interval_mean'] = np.array(tmp_total_interval_mean,dtype=np.float32)



        features['mel1'] = []
        for i in range(len(recordings)) :
            if use_mel :
                mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length,
                                               win_length = win_length, n_mels = n_mels)[0]
            else :
                mel1 = np.zeros( (1,1) )

            features['mel1'].append(mel1)

        M, N = features['mel1'][0].shape

        if use_mel :
            for i in range(len(features['mel1'])) :
                features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

        features['mel1'] = np.array(features['mel1'])

        features['cqt1'] = []
        for i in range(len(recordings)) :
            if use_cqt :
                mel1 = feature_extract_cqt(recordings[i], samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale,
                                            n_bins = n_bins, fmin = fmin)[0]
            else:
                mel1 = np.zeros( (1,1,1) )

            features['cqt1'].append(mel1)

        M, N,__ = features['cqt1'][0].shape

        if use_cqt :
            for i in range(len(features['cqt1'])) :
                features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)

        features['cqt1'] = np.array(features['cqt1'])


        features['stft1'] = []
        for i in range(len(recordings)) :
            if use_stft :
                mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length,
                                            win_length = win_length)[0]
            else :
                mel1 = np.zeros( (1,1,1) )

            features['stft1'].append(mel1)

        M, N,__ = features['stft1'][0].shape
        if use_stft :
            for i in range(len(features['stft1'])) :
                features['stft1'][i] = features['stft1'][i].reshape(M,N,1)
        features['stft1'] = np.array(features['stft1'])

    #     return features

        #    print(features)
        # Impute missing data.
        
        
        
        res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'],
                               features['loc'], features['mel1'], features['stft1'], features['cqt1'],features['interval_mean'],features['interval']])
        res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'],
                               features['loc'], features['mel1'], features['stft1'], features['cqt1'],features['interval_mean'],features['interval']])
        
        if model['ord1'] :
            idx1 = res1.argmax(axis=0)[0]
            murmur_p = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기ddd
            murmur_probabilities = np.zeros((3,))
            murmur_probabilities[0] = murmur_p[0]
            murmur_probabilities[1] = 0
            murmur_probabilities[2] = murmur_p[1]
            outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기

        else :
            if model['mm_mean'] :
                murmur_probabilities = res1.mean(axis = 0)
            else :
                idx1 = res1.argmax(axis=0)[0]
                murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
            outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기


        murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)


        if murmur_probabilities[0] > 0.482 :
            idx = 0
        else :
            idx = 2
        murmur_labels[idx] = 1

        outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
        if outcome_probabilities[0] > 0.607 :
            idx = 0
        else :
            idx = 1
            # idx = np.argmax(outcome_probabilities)
        outcome_labels[idx] = 1

        # Concatenate classes, labels, and probabilities.
        classes = murmur_classes + outcome_classes
        labels = np.concatenate((murmur_labels, outcome_labels))
        probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

        return classes, labels, probabilities
    
output_folder ='/home/jk21/Documents/submission_paper_out/'

In [43]:
data_folder = test_folder

In [44]:
patient_files = find_patient_files(data_folder)
num_patient_files = len(patient_files)

In [45]:
verbose=1

In [46]:
            patient_data = load_patient_data(patient_files[10])
            recordings = load_recordings(data_folder, patient_data)

In [47]:
classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!

ValueError: Layer count mismatch when loading weights from file. Model expected 51 layers, found 99 saved layers.

In [43]:
classes

['Present', 'Unknown', 'Absent', 'Abnormal', 'Normal']